In [1]:
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon
import folium
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from scipy import stats

In [2]:
df=pd.read_csv('merged.csv')

In [3]:
pd.set_option('display.max_columns', None)


In [4]:
df=pd.DataFrame(df)

In [5]:
df= df.drop(df[['transactions', 'price', 'establishment_x', 'establishment_y', 'address_x', 'address_y', 'trimmed_name_x', 'trimmed_name_y', 'clean_address_x', 'clean_address_y', 'address_distance', 'name_distance']], axis=1)

In [6]:
#Dropped all establishments with less than 22 reviews
df=  df[df['review_count'] >= 22]


In [7]:
df.describe()
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1820 entries, 0 to 2408
Data columns (total 17 columns):
Unnamed: 0              1820 non-null int64
inspection_date         1820 non-null object
inspector               1820 non-null object
number_of_violations    1820 non-null int64
violations              1820 non-null object
zip_code                1820 non-null int64
house_numbers           1820 non-null int64
categories              1820 non-null object
distance                1820 non-null float64
review_count            1820 non-null float64
longitude               1820 non-null float64
latitude                1820 non-null float64
name                    1820 non-null object
sector                  1820 non-null int64
district                1820 non-null object
total_score             1820 non-null int64
rating                  1820 non-null float64
dtypes: float64(5), int64(6), object(6)
memory usage: 255.9+ KB


,Unnamed: 0,inspection_date,inspector,number_of_violations,violations,zip_code,house_numbers,categories,distance,review_count,longitude,latitude,name,sector,district,total_score,rating
0,0,2019-01-02,Cicily Martinez,3,"['22', '29', '45']",78212,2524,"['Italian', 'Desserts', 'Seafood']",4950.353662,99.0,-98.493780,29.453730,Capparelli's On Main,35,D,97,3.5
1,1,2019-03-06,RYAN DAVIS,1,['31'],78230,14439,"['French', 'Asian Fusion', 'Tapas/Small Plates']",9312.955170,102.0,-98.544540,29.573330,Frederick's Bistro,9,A,98,4.0
3,3,2019-11-04,Cicily Martinez,3,"['10', '15', '40']",78201,914,['Mexican'],3194.340357,42.0,-98.508619,29.466156,Fina's Kitchen,35,D,93,4.0
4,4,2019-03-14,RYAN DAVIS,0,[],78230,13151,['Barbeque'],7935.733616,41.0,-98.537950,29.562850,Bill Miller Bar-B-Q,9,A,100,2.5
5,5,2019-03-14,RYAN DAVIS,1,['21'],78230,10003,['American (Traditional)'],6996.407916,434.0,-98.536348,29.554391,Green Vegetarian Cuisine,9,A,98,4.5


# START COPYING FROM HERE


In [8]:
#grouped health scores into 4 groups.
health_scores= {'health_scores' : []}
for x in df['total_score']:
#     print(x)
    if x > 99:
        health_scores['health_scores'].append('1')
    elif x>=96:
        health_scores['health_scores'].append('2')
    elif x >=92:
        health_scores['health_scores'].append('3')
    else :
        health_scores['health_scores'].append('4')
#df
df['health_scores']= health_scores['health_scores']  

In [9]:
df['health_scores'].value_counts()

2    523
4    508
3    458
1    331
Name: health_scores, dtype: int64

In [10]:
# confirm sector and inspecor are dependent on each other, i.e. inspectors generally stick to one sector
# this would also indicate the restaurants generally have the same inspectors. We will use that assumption in the model. 
# pd.crosstab(df.inspector, df.sector).T

In [11]:
#encode for the inspectors
df['inspector'].value_counts()

Lauren Schumacher     180
Jeffrey Carrizales    136
RYAN DAVIS            129
Jacob Johanson        128
Sumner Bumgardner     117
Bamba Njie             89
Diana Garcia           88
Deborah Liu            82
Bernard Capdeboscq     79
Cicily Martinez        77
Valerie Suarez         75
Benito Zuniga          72
Daisy Zamora           68
Nathan Balfour         64
Raul Jimenez           38
John Payelle           38
William Hudson         37
Kassandra Aguilar      35
Debra Hernandez        31
Erica Arreola          27
Mario Cavazos          27
MICHELLE MORENO        26
William Emminger       26
Kathleen Prenzler      26
Amanda Cantu           22
Kacie Patrick          20
Adan Zamora            18
Sean Gilbert           16
Fermin Garza           15
Monica Caballero        7
Victoria Paredes        7
Juanicia Page           6
Joe Mokry               5
Maurico Ripley          3
Daniel Guzman           3
Erica Llanas            2
Rebecca Vera            1
Name: inspector, dtype: int64

In [12]:
contingency_table = pd.crosstab(df.inspector, df.health_scores, margins=True)
contingency_table = contingency_table[contingency_table.All > 20]
contingency_table['less_92'] = round(contingency_table['4']/contingency_table['All'],2)
contingency_table['92_to_95'] = round(contingency_table['3']/contingency_table['All'],2)
contingency_table['96_to_99'] = round(contingency_table['2']/contingency_table['All'],2)
contingency_table['100'] = round(contingency_table['1']/contingency_table['All'],2)
contingency_table.reset_index(inplace = True)
contingency_table

health_scores,inspector,1,2,3,4,All,less_92,92_to_95,96_to_99,100
0,Amanda Cantu,3,9,9,1,22,0.05,0.41,0.41,0.14
1,Bamba Njie,1,21,35,32,89,0.36,0.39,0.24,0.01
2,Benito Zuniga,11,18,21,22,72,0.31,0.29,0.25,0.15
3,Bernard Capdeboscq,9,21,21,28,79,0.35,0.27,0.27,0.11
4,Cicily Martinez,10,18,26,23,77,0.30,0.34,0.23,0.13
5,Daisy Zamora,14,36,11,7,68,0.10,0.16,0.53,0.21
6,Deborah Liu,8,34,30,10,82,0.12,0.37,0.41,0.10
7,Debra Hernandez,13,11,3,4,31,0.13,0.10,0.35,0.42
8,Diana Garcia,1,14,30,43,88,0.49,0.34,0.16,0.01
9,Erica Arreola,6,7,7,7,27,0.26,0.26,0.26,0.22


In [13]:
df = df.merge(contingency_table)

In [14]:
df

,Unnamed: 0,inspection_date,inspector,number_of_violations,violations,zip_code,house_numbers,categories,distance,review_count,longitude,latitude,name,sector,district,total_score,rating,health_scores,1,2,3,4,All,less_92,92_to_95,96_to_99,100
0,0,2019-01-02,Cicily Martinez,3,"['22', '29', '45']",78212,2524,"['Italian', 'Desserts', 'Seafood']",4950.353662,99.0,-98.493780,29.453730,Capparelli's On Main,35,D,97,3.5,2,10,18,26,23,77,0.30,0.34,0.23,0.13
1,3,2019-11-04,Cicily Martinez,3,"['10', '15', '40']",78201,914,['Mexican'],3194.340357,42.0,-98.508619,29.466156,Fina's Kitchen,35,D,93,4.0,3,10,18,26,23,77,0.30,0.34,0.23,0.13
2,7,2019-07-05,Cicily Martinez,3,"['1', '39', '45']",78201,919,"['Tex-Mex', 'Breakfast & Brunch']",3138.557809,145.0,-98.508632,29.466663,Rolando's Super Tacos No 1,35,D,95,3.5,3,10,18,26,23,77,0.30,0.34,0.23,0.13
3,8,2019-07-05,Cicily Martinez,3,"['1', '39', '45']",78201,919,"['Tex-Mex', 'Breakfast & Brunch']",3138.557809,145.0,-98.508632,29.466663,Rolando's Super Tacos No 1,35,D,95,3.5,3,10,18,26,23,77,0.30,0.34,0.23,0.13
4,10,2019-07-05,Cicily Martinez,3,"['9', '37', '40']",78201,914,['Mexican'],3194.340357,42.0,-98.508619,29.466156,Fina's Kitchen,35,D,95,4.0,3,10,18,26,23,77,0.30,0.34,0.23,0.13
5,15,2019-11-04,Cicily Martinez,4,"['30', '37', '41', '45']",78201,919,"['Tex-Mex', 'Breakfast & Brunch']",3138.557809,145.0,-98.508632,29.466663,Rolando's Super Tacos No 1,35,D,97,3.5,2,10,18,26,23,77,0.30,0.34,0.23,0.13
6,16,2019-11-04,Cicily Martinez,4,"['30', '37', '41', '45']",78201,919,"['Tex-Mex', 'Breakfast & Brunch']",3138.557809,145.0,-98.508632,29.466663,Rolando's Super Tacos No 1,35,D,97,3.5,2,10,18,26,23,77,0.30,0.34,0.23,0.13
7,32,2019-10-25,Cicily Martinez,0,[],78212,1906,"['Pizza', 'Salad', 'Italian']",5594.631902,214.0,-98.493900,29.447410,Main Street Pizza & Pasta,35,D,100,4.0,1,10,18,26,23,77,0.30,0.34,0.23,0.13
8,42,2019-03-21,Cicily Martinez,1,['31'],78212,3233,"['Coffee & Tea', 'Coffee Roasteries']",5303.393739,208.0,-98.483310,29.454800,Summer Moon Coffee Bar,35,D,100,4.5,1,10,18,26,23,77,0.30,0.34,0.23,0.13
9,71,2019-02-25,Cicily Martinez,0,[],78201,919,"['Tex-Mex', 'Breakfast & Brunch']",3138.557809,145.0,-98.508632,29.466663,Rolando's Super Tacos No 1,35,D,100,3.5,1,10,18,26,23,77,0.30,0.34,0.23,0.13


In [15]:
df = df[['name','review_count','rating','less_92','92_to_95','96_to_99','100', 'health_scores']]
df.head()

,name,review_count,rating,less_92,92_to_95,96_to_99,100,health_scores
0,Capparelli's On Main,99.0,3.5,0.3,0.34,0.23,0.13,2
1,Fina's Kitchen,42.0,4.0,0.3,0.34,0.23,0.13,3
2,Rolando's Super Tacos No 1,145.0,3.5,0.3,0.34,0.23,0.13,3
3,Rolando's Super Tacos No 1,145.0,3.5,0.3,0.34,0.23,0.13,3
4,Fina's Kitchen,42.0,4.0,0.3,0.34,0.23,0.13,3


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [17]:
#training / test split
X= df.drop(columns=['name','health_scores'])
y= df['health_scores']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state = 42, stratify=df.health_scores)
X_train.head()

,review_count,rating,less_92,92_to_95,96_to_99,100
549,361.0,4.5,0.30,0.20,0.38,0.12
1063,27.0,3.5,0.26,0.19,0.21,0.34
456,445.0,3.5,0.06,0.23,0.52,0.19
1520,93.0,3.0,0.11,0.27,0.32,0.30
1319,362.0,3.0,0.57,0.26,0.14,0.02


In [18]:
y_train.head()

549     3
1063    4
456     4
1520    3
1319    4
Name: health_scores, dtype: object

In [19]:
#Train Model

#Create the Decision Tree Object

clf = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state= 42, min_samples_split= 2)

In [20]:
#Fit the model to the training data

clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

In [21]:
#Estimate health group

y_pred = clf.predict(X_train)
y_pred

array(['4', '4', '2', ..., '2', '2', '4'], dtype=object)

In [22]:
#Estimate the probability of a health group
y_pred_proba = clf.predict_proba(X_train)
y_pred_proba

array([[0.08024691, 0.24691358, 0.2962963 , 0.37654321],
       [0.26315789, 0.19298246, 0.24561404, 0.29824561],
       [0.06818182, 0.56818182, 0.25      , 0.11363636],
       ...,
       [0.17924528, 0.39622642, 0.17924528, 0.24528302],
       [0.06666667, 0.93333333, 0.        , 0.        ],
       [0.        , 0.04545455, 0.45454545, 0.5       ]])

In [23]:
confusion_matrix(y_train, y_pred)

array([[111,  69,   2,  38],
       [ 61, 161,  19,  96],
       [ 31,  90,  33, 152],
       [ 11,  52,  24, 251]])

In [24]:
#Accuracy: number of correct predictions over the number of total instances that have been evaluated.
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))

Accuracy of Decision Tree classifier on training set: 0.46


In [25]:
#Test Model

#Compute the accuracy of the model when run on the test data

print('Accuracy of Decision Tree classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))

Accuracy of Decision Tree classifier on test set: 0.41


In [26]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           1       0.52      0.50      0.51       220
           2       0.43      0.48      0.45       337
           3       0.42      0.11      0.17       306
           4       0.47      0.74      0.57       338

    accuracy                           0.46      1201
   macro avg       0.46      0.46      0.43      1201
weighted avg       0.46      0.46      0.43      1201



In [27]:
from sklearn.ensemble import RandomForestClassifier

In [28]:
rf = RandomForestClassifier(max_depth=4, criterion='entropy', n_estimators=1000000, random_state=123)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=4, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000000,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [29]:
y_pred = rf.predict(X_train)

In [30]:
rf.score(X_train, y_train)

0.46378018318068276

In [ ]:
rf.score(X_test, y_test)